# Define file and what sheet should be used for fit 
<span style="color:green">do changes here</span>.


The first step in the fitting process is getting the data.
It is assumed that data is always of the type as Neil gave, in an xlsx file with different sheets.

The first row has the names of the different data sets. 
and each further row has the corresponding data points for each set.

In the data, 2 consecutive sheets correspond to the same experiment and thus it is sufficient to consider only every other.


In [ ]:
#Choice of parameters
#"../231013_DtxR_data_Michael"
filename = "../230920_data_for_Michael" #xlsx file that contains all data in the different sheets
#extract the data to analyse from the xlsx file

#choose a sheet that is being analysed in the following, structure assumed as  Neil's xlsx file
sheetNr = 16
#TrpR: [2,10,12]   = [#5, #13, #22]
#CoCl:  [4,6,8,14,16] = [#8,#4,#10,#24,#7]

#Not all concentrations in the file may be interesting, the following integer decides how many concentrations are 
#being ignored. With the current order in the files, this typically means low concentrations are being ignored, which
#often are too low and have negative responses, which can't be fitted
number_ignored_motor_concentrations = 4  #   > 0,    


#in principle the program should be able to have multiple washing cycles different than 2
#however, changing this number here will mean that the program expects more parameters later for the fitting.
#without changing that later too, the program will probably break
#so do not change the number of washes from 2, unless one knows what one has to change later.
nrWashings = 2 #define the number of washings used

### Tips for using different number of washings:  ####
###################################################
#if one wants to change the number washings, then one has to do straightforward changes in the pre-processing stage 
#where the cuts need more arguments. That is explained there in the comments.
# the fitting process is more difficult to change, since it needs more variables that have to be passed (if more washings are considered), in particular the fit functions need more arguments
#i tried to automate that, but did not find out how to make kwargs work with lmfit...
#so one would have to rewrite the 'helper_functions.collective_spr_fit_constant_k23' and 'helper_functions.collective_spr_fit_linear_k23'
# to have more arguments and use them. One also woud need to define these new parameters in the paramsdict for the linear and constant case




## Getting the data from the file
<span style="color:red">(typically no changes needed here)</span>.


Here the data is read out of the xlsx file and stored for later usage

One can also see here the names of the data showing e.g. what concentrations were used.

In [ ]:
#The functions needed for getting the data from the sheets in Neil's format are stored in here
import python_scripts.xlsx_file_read_extractor as xlsx_extractor
#Here are some helper functions that are useful for the program
import python_scripts.Collective_fit_functions as helper_functions
#Here are the solutions for the rate equation problem with multiple washes
import python_scripts.Rate_equation_solution as rateEqs
import numpy as np
from pandas import Series
# from lmfit import Model, Parameter, report_fit
import lmfit
import matplotlib.pyplot as plt

#The workbook that contains the full xlsx file
used_workbook = xlsx_extractor.Get_workbook_from_xlsx_file(filename, True)
#with the prior given sheetNr, here one gets the actual sheet from the workbook
usedSheet = used_workbook[used_workbook.sheetnames[sheetNr]]

#Get all the data entries from the sheet in a dict format. With the current format of Neil's file, the first row is ignored 
#and the second one is given as dict-kwvals which each connect to the rest of its row
dataDict = xlsx_extractor.Get_data_from_sheet(usedSheet) 
#store all keys which correspond to the motor concentrations-data location and where the time is in the dict
allkeys = list(dataDict.keys()) 


#output the data from the sheet for reference
print('name of the used sheet:')
print(usedSheet.title)
print('found key names:')
for x in allkeys:
    print(x)


# Visualization plot of the received data
<span style="color:red">(no changes needed here)</span>.


Use the plot to consider what should be cut out from the data
Especially log data is helpful, to look for negative values

In [ ]:
timeKey = allkeys[0]
usedDataKeys = allkeys[number_ignored_motor_concentrations:]
# the numbers used for getting all the motor concentrations in nM
motor_concentration=usedDataKeys 


if number_ignored_motor_concentrations != 1:
    print('Note: not all data is being used')
maxy = 0
timevals = dataDict[timeKey]
yvalslist = []
plt.rcParams.update({'font.size': 12})

for x in usedDataKeys:
    yvals = dataDict[x]
    if maxy < max(yvals):
        maxy = max(yvals)
    yvalslist.append(yvals)    

fig, ax_dict = plt.subplot_mosaic([['vis']],figsize=[6,4])

for (x,y) in zip(reversed(yvalslist),reversed(usedDataKeys)):
    data = x
    times = timevals
    ax_dict['vis'].plot(timevals,  data , 'x-',markersize=1,lw=1,  label=y)

ax_dict['vis'].set_ylabel('RU')
ax_dict['vis'].set_xlabel(timeKey)

#uncomment below to plot log scale
# ax_dict['vis'].set_yscale('log')
 
plt.legend(title='motor (nM)')

# Data pre-processing step
<span style="color:yellow">(do sometimes changes here)</span>.

Data thinning, finding of the peaks and cutting the peaks out is being done here. 

Choose good cut offs to get the best data


Further, it is needed to cut off negative responses at the peak positions, as otherwhise fitting is not possible.

One can also cut off the total length of the run, if e.g. the 2. wash is too long and nothing interesting happens anymore.

Has to be changed only sometimes, depending on the response behavior, but most of the time should be ok as it is

In [ ]:
################################################
################################################
#vary parameters

#define the time in s when data should no longer be considered.
# might be useful to cut off very long 2. washes.
endTimeCutoff = 100

#define how many data points the washing peaks should be cut off, 
#after a washing peak is registered with my program  i.e. peakposition + offset
#The numbers correspond to individual data points, not time!, 
#so e.g. [1,1] is cutting away 1 point after each of two found peaks
#!! it is always needed that: len(after_peak_offset) = # peaks
after_peak_offset = [17,29]   

#define how much before a registered peak should be considered
#same idea as with start cutoff, just for individual points before the registered peak position
#Since the first peak is used as the start of simulation, one has to define
#! len(before_peak_offset) = len(after_peak_offset) - 1  
before_peak_offset = [10]     

#approximate number of data points
#the program will thin out the given data to have so many points in the curve of a single motor concentration
#the full data can't be fitted, so this reduces the data for fitting
numberOfDataPoints = 50

################################################
################################################

#define the key which corresponds to the time in the data dict
timeKey = allkeys[0]
# here one can supply all keys that one is interested in as data
#here it is also evident that number_ignored_motor_concentrations needs to be >0, as it is assumed that 0 is the time
usedDataKeys = allkeys[number_ignored_motor_concentrations:]

#now the data is being processed here
yvalsdictall = dict()
timevalall = []
t0_estimates = []
yvalsdictall, timevalall, t0_estimates = helper_functions.cut_peaks_and_shrink_data(dataDict, timeKey, usedDataKeys,nrWashings, endRunTime = endTimeCutoff,offsetBeforePeak = before_peak_offset,offsetAfterPeak = after_peak_offset,nrOfDataPoints=numberOfDataPoints)

# here the washing estimates is being printed
print('found times of washings')
print(t0_estimates)

#consolidate all data into a single stacked vector which is needed for the fitting process
data=np.array([])
for x in usedDataKeys:
    plt.plot(timevalall,yvalsdictall[x], label=x, marker='o',markersize=4)
    data=np.hstack([data,yvalsdictall[x]])


plt.legend(title='[TW] in nM')    
plt.xlabel(timeKey)
plt.ylabel('RU')
plt.yscale('log')
plt.title(f'Data of exp. {usedSheet.title}')
#some misc. stuff if one wants to modify the plotting/ saving it

# plt.savefig('Full_data_exp_' + usedSheet.title + '.pdf' )
# plt.ylim([0,maxy])
#choose to show everything in log scale
# plt.yscale('log')
# plt.xlim([0,dataDict[timeKey][-1]])
# plt.savefig('Data_' + usedSheet.title + ".pdf")

# Collective fit

## Linear fit for $k_{23}$

### Initial state finding
<span style="color:green">(do changes here)</span>.

Find here a good choice for the initial fit guess.

As the problem is highly nonlinear, it is vital to start the fit with a good guess.

For this stage, one can only brute force a good guess.

The program will always plot your guess, so you can just vary until it looks somewhat close.

### Difficulties:

* It can happen, that if one chooses poor values, that the following fit creates 'NaN'-values which it says it cannot handle.
This might be because of the strong exponential nature of all terms involved, that for long times and bad k-choices the responses will reach 0 and then the log of it becomes NaN. 
So if that happens one has to choose different start values.
* Besides not well (or at all) converging if poor starting values were used, the lmfit will then also be quite slow (~30-40 minutes) if the choice is too far away.

In [ ]:
#define here the fit function
linear_fit_function = lambda t,C1,C2,C3,a_k12_1,a_k12_2,k21,a_k23_1,a_k23_2,k32,t0_1,t0_2,weight_C3: helper_functions.collective_spr_fit_linear_k23(t=t,C1=C1,C2=C2,C3=C3,a_k12_1=a_k12_1,a_k12_2=a_k12_2,k21=k21,a_k23_1=a_k23_1,a_k23_2=a_k23_2,k32=k32,t0_1=t0_1,t0_2=t0_2,motor_concentrations=motor_concentration,weight_C3=weight_C3)
#fitparameteres

paramsdict = dict()
paramsdict['C1'] = {'value':14.5, 'min': 1, 'max': 210, 'vary':True}
paramsdict['k32'] = {'value':1.5e-5, 'min': 1e-10, 'max': 0.1, 'vary':True}
paramsdict['k21'] = {'value':0.051, 'min': 1e-10, 'max': 1, 'vary':True}
paramsdict['a_k12_1'] = {'value':0.0028, 'min': 1e-10, 'max': 5, 'vary':True}
paramsdict['a_k23_1'] = {'value':4e-4, 'min': 1e-10, 'max': 5,  'vary':True}
paramsdict['a_k12_2'] = {'value':0.0, 'vary':False}
paramsdict['a_k23_2'] = {'value':0.0, 'vary':False}
paramsdict['C2'] = {'value':0, 'vary':False}#finite for log fit, as otherwise response would diverge log(0)
paramsdict['C3'] = {'value':0.00000001, 'vary':False}
paramsdict['t0_1'] = {'value':t0_estimates[0],'min':-10,'max':1.01, 'vary':True}
paramsdict['t0_2'] = {'value':t0_estimates[1],'min':t0_estimates[1]*0.95,'max':t0_estimates[1]*1.05, 'vary':True}
paramsdict['weight_C3'] = {'value':2,'vary':False}

# mulitplicator = 1
# paramsdict['k21']['value'] *= mulitplicator
# paramsdict['a_k12_1']['value'] *= mulitplicator
# #fitting the data with linear k23 dependence
# resultLinear = fit_spr_data_to_model(paramsdict,timevalall,np.log(data),collective_spr_fit_linear_k23, True)


multiplicator = 4
paramsdict['k21']['value'] *= multiplicator
paramsdict['a_k12_1']['value'] *= multiplicator
# plot the initial parameters to see how well the starting state is defined
helper_functions.fit_spr_data_to_model(fitparams=paramsdict,fitdata=np.log(data),times=timevalall,fitfunction=linear_fit_function,concentrations=motor_concentration, only_show_start_fit=True)


### Linear fit
<span style="color:red">(typically no changes needed)</span>.

Here the data is fitted to the linear model using the above initial values and then plotted as well as saved as a pdf with an automatically generated name consisting of the type of fit and which data it belongs to.

One can change the name of the pdf here partially by supplying another string to the plot_and_save_result funciton

In [ ]:
#fit linear data
resultLinear = helper_functions.fit_spr_data_to_model(fitparams=paramsdict,fitdata=np.log(data),times=timevalall,fitfunction=linear_fit_function,concentrations=motor_concentration)

#plot the result
helper_functions.plot_and_save_result(resultLinear,data,timevalall,usedSheet.title + '_linear_fit',motor_concentration)

### Output the result of the fit parameters

<span style="color:red">(no changes needed)</span>

Here the fit result is plotted and their erros given.

If the fit is too poor lmfit is not able to define errors, so that is another indicator that the fit did not work out properly.

One can copy the result from here, or write a script that saves this data somewhere else.

In [ ]:
resultLinear[0]

## Fit of constant $k_{23}$ dependence

### Choice of initial parameters 

<span style="color:green">(do changes here)</span>

Here the constant fit initial values are being defined.

As a simple start, the values of the linear fit are copied, as they both models should be relatively similar.

Then changes are made to fit the differing behavior, like 'a_k23' being here just $k_{23}$, a constant, instead of the linear prefactor of $k_{23} = a_{k_{23}} C_\text{motor}$ like in the linear fit.

Further, the free DNA concentration 'C1' will be different, as with the lower weight of 'C3', one needs generally more DNA to be able to fit the data.

In [ ]:
import copy
#define here the fit function
constant_fit_function = lambda t,C1,C2,C3,a_k12_1,a_k12_2,k21,a_k23_1,a_k23_2,k32,t0_1,t0_2,weight_C3: helper_functions.collective_spr_fit_constant_k23(t=t,C1=C1,C2=C2,C3=C3,a_k12_1=a_k12_1,a_k12_2=a_k12_2,k21=k21,a_k23_1=a_k23_1,a_k23_2=a_k23_2,k32=k32,t0_1=t0_1,t0_2=t0_2,motor_concentrations=motor_concentration,weight_C3=weight_C3)

#the parameters for the constant fit
paramsdictconstant = copy.deepcopy(paramsdict)

paramsdictconstant['weight_C3'] = {'value':1,'vary':False}
paramsdictconstant['a_k12_1'] = {'value':0.023, 'min': 1e-10, 'max': 5, 'vary':True}
# paramsdictconstant['k21'] = {'value':0.78, 'min': 1e-10, 'max': 5, 'vary':True}

paramsdictconstant['a_k23_1'] = {'value':0.005, 'min': 1e-10, 'max': 5,  'vary':True}
paramsdictconstant['C1'] = {'value':12.5, 'min': 1, 'max': 210, 'vary':True}

#keeping the ratio of the rates constant, but multiplying them by a factor allows for the same steady state responses
#but change the speed with which the system thermalizes
multiplicator = 1
paramsdictconstant['k21']['value'] *= multiplicator
paramsdictconstant['a_k12_1']['value'] *= multiplicator
helper_functions.fit_spr_data_to_model(paramsdictconstant,timevalall,np.log(data),constant_fit_function,motor_concentration, True)


### Constant fit
<span style="color:red">(no changes needed)</span>

Here the constant fit is being performed using the above defined constant initial parameters

In [ ]:

#fitting the data with constant k23 dependence (a_k23_1 is uased as k_23)
resultConstant = helper_functions.fit_spr_data_to_model(paramsdictconstant,timevalall,np.log(data),constant_fit_function,motor_concentration)
helper_functions.plot_and_save_result(resultConstant,data,timevalall,usedSheet.title + '_constant_fit',motor_concentration)

### Plot of the fit results for the parameters of the constant fit

<span style="color:red">(typically no changes needed)</span>

Here the fit parameter results are given with their errors for the constant case, which can be copied.

In [ ]:
resultConstant[0]

# Ideal data plotting and fitting

Here one can see how to generate and fit to ideal data.

Just uncomment the following two cells

This might be helpful to get a better feeling on how to use the lmfit and how well it should be able to fit to the data.


In [ ]:
# #define here the fit function

# motor_concentration = [0.01,0.1,1,5]

# #the linear dependence on k23
# linear_data_function = lambda t,C1,C2,C3,a_k12_1,a_k12_2,k21,a_k23_1,a_k23_2,k32,t0_1,t0_2,weight_C3: helper_functions.collective_spr_fit_linear_k23(t=t,C1=C1,C2=C2,C3=C3,a_k12_1=a_k12_1,a_k12_2=a_k12_2,k21=k21,a_k23_1=a_k23_1,a_k23_2=a_k23_2,k32=k32,t0_1=t0_1,t0_2=t0_2,motor_concentrations=motor_concentration,weight_C3=2)
# #the constant dependence on k23
# constant_data_function = lambda t,C1,C2,C3,a_k12_1,a_k12_2,k21,a_k23_1,a_k23_2,k32,t0_1,t0_2,weight_C3: helper_functions.collective_spr_fit_constant_k23(t=t,C1=C1,C2=C2,C3=C3,a_k12_1=a_k12_1,a_k12_2=a_k12_2,k21=k21,a_k23_1=a_k23_1,a_k23_2=a_k23_2,k32=k32,t0_1=t0_1,t0_2=t0_2,motor_concentrations=motor_concentration,weight_C3=1)

# #fitparameteres

# times = np.linspace(0,170,40)
# t0_estimates = [-0.1,50]
# paramsdict = dict()
# paramsdict['C1'] = {'value':14.5, 'min': 1, 'max': 210, 'vary':True}
# paramsdict['k32'] = {'value':1.5e-3, 'min': 1e-10, 'max': 0.1, 'vary':True}
# paramsdict['k21'] = {'value':0.051, 'min': 1e-10, 'max': 1, 'vary':True}
# paramsdict['a_k12_1'] = {'value':0.0028, 'min': 1e-10, 'max': 5, 'vary':True}
# paramsdict['a_k23_1'] = {'value':4e-4, 'min': 1e-10, 'max': 5,  'vary':True}
# paramsdict['a_k12_2'] = {'value':0.0, 'vary':False}
# paramsdict['a_k23_2'] = {'value':0.0, 'vary':False}
# paramsdict['C2'] = {'value':0, 'vary':False}#finite for log fit, as otherwise response would diverge log(0)
# paramsdict['C3'] = {'value':0.00000001, 'vary':False}
# paramsdict['t0_1'] = {'value':t0_estimates[0],'min':-10,'max':1.01, 'vary':True}
# paramsdict['t0_2'] = {'value':t0_estimates[1],'min':t0_estimates[1]*0.95,'max':t0_estimates[1]*1.05, 'vary':True}
# paramsdict['weight_C3'] = {'value':2,'vary':False}
# import lmfit
# parametersdict = dict()
# for x in paramsdict:
#     parametersdict[x] = paramsdict[x]['value']
# idealdata = np.exp(linear_data_function(times, **parametersdict))

# import matplotlib.pyplot as plt
# for (y,z) in zip(np.reshape(idealdata,(len(motor_concentration),len(times))),motor_concentration):
#         plt.plot(times,y, '--', label=z)

In [ ]:
# #fit linear data
# #choose some different values for the initial point of the fit process
# #depending how far away, it will be able to fit perfectly
# paramsdict['C1']['value'] = 20
# paramsdict['a_k12_1']['value'] = 0.001

# #show initial parameter choice
# parametersdict = dict()
# for x in paramsdict:
#     parametersdict[x] = paramsdict[x]['value']
# initialGuess = linear_data_function(times, **parametersdict)

# import matplotlib.pyplot as plt
# for (y,z,k) in zip(np.reshape(idealdata,(len(motor_concentration),len(times))),motor_concentration, np.reshape(initialGuess,(len(motor_concentration),len(times)))):
#         plt.plot(times,np.log(y), '--', label=z)
#         plt.plot(times,k, '*')
# plt.legend()
# plt.xlabel('time in s')
# plt.title('initial guess')
# plt.ylabel('log(response)')
# plt.show()
# fitideal = helper_functions.fit_spr_data_to_model(fitparams=paramsdict,fitdata=np.log(idealdata),times=times,fitfunction=linear_data_function,concentrations=motor_concentration)

# #plot the result
# helper_functions.plot_and_save_result(fitideal,idealdata,times,usedSheet.title + '_ideal_fit',motor_concentration)